In [2]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Define the directory containing the PDF files
pdf_directory = './data'

### 논문을 벡터 db에 넣기

In [3]:
# pdf를 사용해서 pdf(논문)을 모두 로드
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Load all PDF files using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)
    
# 텍스트는 RecursiveCharacterTextSplitter를 사용하여 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

# embeddings은 OpenAI의 임베딩을 사용
# vectordb는 chromadb사용함

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

### 인적정보 가져오기

In [4]:
import json

class User:
    def __init__(self, info_dir):
        # Read the JSON file
        with open(info_dir, 'r') as file:
            data = json.load(file)
        self.name = data['name']
        self.phone = data['phone']
        self.age = data['age']
        self.gender = data['gender']
        self.education = data['education']
        self.merry = data['merry']
        self.children = data['children']
        self.religion = data['religion']
        self.income = data['income']
        self.economy_states = data['economy_states']
        self.health_states = data['health_states']
        
# Specify the path to the JSON file
info_dir = 'data/user_info.json'
user = User(info_dir)
print(user.name)

김영수


### KGLS 질문/답변 가져오기

In [14]:
import json

class KGLSData:
    def __init__(self, file_path):
        # JSON 파일 읽기
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        self.kgls = data['KGLS']
        
    def get_question(self, q_num):
        for item in self.kgls:
            if item['q_num'] == q_num:
                return item['question']
        return None
    
    def get_user_choice(self, q_num):
        for item in self.kgls:
            if item['q_num'] == q_num:
                return item['user_choose']
        return None
    
    def get_all_questions(self):
        return [item['question'] for item in self.kgls]
    def get_all_user_choices(self):
        return [item['user_choose'] for item in self.kgls]
    def get_all_user_choices_sum(self):
        return sum([item['user_choose'] for item in self.kgls])


# 파일 경로 지정
file_path = './data/kgls_dummy.json'
kgls_data = KGLSData(file_path)

# 예시 출력
print(kgls_data.get_question(1))       # "나는 가족들과 매일 대화를 나눈다."
print(kgls_data.get_user_choice(1))    # 3
print(kgls_data.get_all_questions())   # 모든 질문 리스트 출력


나는 가족들과 매일 대화를 나눈다.
3
['나는 가족들과 매일 대화를 나눈다.', '나는 가깝게 지내는 사람들이 있다.', '나는 쓸모없는 사람이라고 느껴진다.', '나를 이해해주는 사람이 있다.', '사람들은 나와 겉으로만 어울리는 것 같다.', '나는 자녀에게 고민을 이야기할 수 있다.', '나는 주변 사랑과의 관계가 만족스럽다.', '가족들은 나를 예전처럼 대해주지 않는다.', '친구나 이웃들은 나에게 관심을 둔다.', '내가 아플 때 나를 보살펴줄 가족이 있다.', '나는 의지할 친구가 있다.', '나는 온종일 할 일없이 시간을 보낸다.', '나는 사회에서 필요한 사람이다.', '나는 가족에게 의지할 수 있다.']


### 프롬프트

In [59]:
OUTPUT_FORMAT=f"""
###노인 외로움 측정 모델 \\
KGLS 14문항은 likert 4점 척도로 1점~4점 사이의 값으로 사용자의 답변을 받습니다.\n \\
KGLS 14문항 점수 : {kgls_data.get_all_user_choices_sum()}점 \n\\
{user.name}님은 {kgls_data.get_all_user_choices_sum()} 점으로 아래의 기준에 따라 ~~한 외로움을 가지고 있습니다 \n \\
###외로움 구분 점수 기준 \\
14점 ~ 24점: 매우 높은 외로움 \n\\
25점 ~ 35점: 높은 외로움 \n \\
36점 ~ 46점: 중간 정도의 외로움\n \\
47점 ~ 56점: 낮은 외로움\n \\
"""

In [65]:
SYS_PROMPT = f"""
    사용자의 인적정보, KGLS 질문과 한국형 외로움에 대한 연구(KGLS 논문)를 바탕으로 어떤 한국형 외로움을 가지고있는지 설명해주세요.\\
    답변 서두의 양식은 {OUTPUT_FORMAT}과 같습니다. \\
    ### 사용자의 외로움
    그 뒤에는 논문을 바탕으로 사용자의 한국형 외로움에 대해 해석해주세요. \\
    ### 사용자가 취약한 외로움
    사용자가 취약한 외로움에 대해서 알려주세요. 인적사항에 대해 큰 가중치를 부여해주세요.   \\
    ### 해결방안
    사용자가 할 수 있는 해결방안을 해당 논문들을 참고하여 답변해주세요. \\
    KGLS data format\\
    {kgls_data.get_all_questions()} 이게 KGLS질문이에요.\\
    {kgls_data.get_all_user_choices()} 이게 KGLS질문에 대한 사용자의 답변이에요.\\
    사용자 인적 정보\\
    사용자의 이름은 {user.name}, 나이는 {user.age}, 성별은 {user.gender}, 학력은 {user.education}, 결혼 여부는 {user.merry}, 자녀 수는 {user.children}, 종교는 {user.religion}, 소득은 {user.income}, 경제 상태는 {user.economy_states}, 건강 상태는 {user.health_states}입니다. \\
    
"""

INPUT_PROMPT = f"""
라고 대답했습니다. \\
이 사용자가 가지고 있는 한국형 외로움을 설명해주세요. \\
그리고 어떤 대화 상대가 되어주어야 하는지 출력해주세요. \\
"""

모델 선언

In [66]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.
template = SYS_PROMPT + '''Answer the question based only on the following context:
{context}

Question: {question}
'''

# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.
input_message =  """
Q: 어떤 상황에서 외로움을 느끼시나요? \\
A: 가장 외로웠던 상황은 가족들과 떨어져 살았을 때입니다. 내가 젊을 때 기러기 아빠였어요. 밤에 혼자 집에 가는데 집에 가도 아무도 없겠구나 싶어서 힘들더라고요. 지금도 가끔 생각나요. \\
""" + INPUT_PROMPT  # 추가적인 입력 프롬프트가 이어집니다.

# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
to_markdown(rag_chain.invoke("input_message"))


> ### 노인 외로움 측정 모델
> KGLS 14문항은 likert 4점 척도로 1점~4점 사이의 값으로 사용자의 답변을 받습니다.
> 
> KGLS 14문항 점수 : 42점
> 
> 김영수님은 42 점으로 아래의 기준에 따라 중간 정도의 외로움을 가지고 있습니다.
> 
> ### 외로움 구분 점수 기준
> 14점 ~ 24점: 매우 높은 외로움
> 
> 25점 ~ 35점: 높은 외로움
> 
> 36점 ~ 46점: 중간 정도의 외로움
> 
> 47점 ~ 56점: 낮은 외로움
> 
> ### 사용자의 외로움
> 김영수님은 65세의 남성으로, 고등학교를 졸업하였으며 결혼하여 자녀가 한 명 있습니다. 종교가 있으며, 소득은 100,000원으로 경제 상태는 보통이지만 건강 상태는 좋지 않습니다. 이러한 인적 사항을 바탕으로 볼 때, 김영수님의 외로움은 주로 건강 상태와 관련이 있을 가능성이 큽니다. 건강이 좋지 않으면 사회적 활동이 제한되고, 이는 외로움을 증가시킬 수 있습니다.
> 
> ### 사용자가 취약한 외로움
> 김영수님은 건강 상태가 좋지 않기 때문에 신체적 활동이 제한될 수 있으며, 이는 사회적 고립을 초래할 수 있습니다. 또한, 경제 상태가 보통이지만 소득이 낮은 편이기 때문에 경제적 스트레스도 외로움을 가중시킬 수 있습니다. 종교가 있는 점은 긍정적이지만, 종교 활동에 참여할 수 있는 기회가 제한될 경우 외로움을 느낄 수 있습니다.
> 
> ### 해결방안
> 1. **건강 관리**: 건강 상태가 외로움에 큰 영향을 미치므로, 정기적인 건강 검진과 적절한 치료를 받는 것이 중요합니다. 또한, 가능한 한 신체 활동을 유지하는 것이 좋습니다. 예를 들어, 가벼운 산책이나 스트레칭을 통해 신체적 활동을 유지할 수 있습니다.
> 
> 2. **사회적 활동 참여**: 종교 활동이나 지역 사회의 모임에 적극적으로 참여하는 것이 좋습니다. 이는 사회적 연결을 강화하고 외로움을 줄이는 데 도움이 됩니다.
> 
> 3. **가족과의 소통**: 자녀와의 소통을 늘리고, 가족과의 시간을 더 많이 보내는 것이 중요합니다. 가족과의 정기적인 대화와 만남을 통해 정서적 지지를 받을 수 있습니다.
> 
> 4. **경제적 지원**: 경제적 스트레스를 줄이기 위해 가능한 지원 프로그램을 활용하는 것이 좋습니다. 예를 들어, 정부나 지역 사회에서 제공하는 경제적 지원 프로그램을 찾아보는 것이 도움이 될 수 있습니다.
> 
> 5. **심리적 지원**: 필요하다면 전문가의 도움을 받아 심리적 지원을 받는 것도 고려해볼 만합니다. 상담을 통해 외로움과 관련된 감정을 다루는 데 도움이 될 수 있습니다.
> 
> 이와 같은 방안을 통해 김영수님의 외로움을 줄이고, 더 나은 삶의 질을 유지할 수 있을 것입니다.